In [6]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()
LENGTH_SEQ = 5


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [9]:
class DataGenerator(object):
    def next_batch(self, batch_size, N, train_mode=True):
        """Return the next `batch_size` examples from this data set."""

        # A sequence of random numbers from [0, 1]
        encoder_batch = []

        # Sorted sequence that we feed to encoder
        # In inference we feed an unordered sequence again
        decoder_batch = []

        # Ordered sequence where one hot vector encodes
        # position in the input array
        target_batch = []
        for _ in range(batch_size):
            encoder_batch.append(np.zeros([N, 1]))
        for _ in range(batch_size):
            decoder_batch.append(np.zeros([N, 1]))
            target_batch.append(np.zeros([N, N]))

        encoder_batch = np.asarray(encoder_batch)
        decoder_batch = np.asarray(decoder_batch)
        target_batch = np.asarray(target_batch)

        for b in range(batch_size):
            shuffle = np.random.permutation(N)
            sequence = np.sort(np.random.random(N))
            shuffled_sequence = sequence[shuffle]

            for i in range(N):
                encoder_batch[b][i] = shuffled_sequence[i]
                if train_mode:
                    decoder_batch[b][i] = sequence[i]
                else:
                    decoder_batch[b][i] = shuffled_sequence[i]
                target_batch[b, i][shuffle[i]] = 1.0

            # Points to the stop symbol
            #target_batch[b, N][0] = 1.0

        return encoder_batch, decoder_batch, target_batch

In [1]:
import random
import itertools
import math
import numpy as np

LENGTH_SEQ = 10

class Tsp:
    def next_batch(self, batch_size=1):
        X, Y = [], []
        for b in range(batch_size):
            #print("preparing dataset... %s/%s" % (b, batch_size))
            points = self.generate_data()
            solved = self.solve_tsp_dynamic(points)
            X.append(points), Y.append(solved)
        return np.asarray(X), np.asarray(Y)

    def length(self, x, y):
        return (math.sqrt((x[0]-y[0])**2 + (x[1]-y[1])**2))

    def solve_tsp_dynamic(self, points):
        # calc all lengths
        all_distances = [[self.length(x, y) for y in points] for x in points]
        # initial value - just distance from 0 to
        # every other point + keep the track of edges
        A = {(frozenset([0, idx+1]), idx+1): (dist, [0, idx+1])
             for idx, dist in enumerate(all_distances[0][1:])}
        cnt = len(points)
        for m in range(2, cnt):
            B = {}
            for S in [frozenset(C) | {0}
                      for C in itertools.combinations(range(1, cnt), m)]:
                for j in S - {0}:
                    B[(S, j)] = min([(A[(S-{j}, k)][0] + all_distances[k][j],
                                      A[(S-{j}, k)][1] + [j])
                                     for k in S if k != 0 and k != j])
            A = B
        res = min([(A[d][0] + all_distances[0][d[1]], A[d][1])
                   for d in iter(A)])
        return res[1]

    def generate_data(self, N=LENGTH_SEQ):
        radius = 1
        rangeX = (0, 10)
        rangeY = (0, 10)
        qty = N

        deltas = set()
        for x in range(-radius, radius+1):
            for y in range(-radius, radius+1):
                if x*x + y*y <= radius*radius:
                    deltas.add((x, y))

        randPoints = []
        excluded = set()
        i = 0
        while i < qty:
            x = random.randrange(*rangeX)
            y = random.randrange(*rangeY)
            if (x, y) in excluded:
                continue
            randPoints.append((x, y))
            i += 1
            excluded.update((x+dx, y+dy) for (dx, dy) in deltas)
        return randPoints

In [94]:
dataset = Tsp()

#Returns tuple of 3 elements: inputs, inputs ordered acc. to solution, solution in one-hot encoding
def generate_tsp_batch(batch_size=32):

    input_pts, soln = dataset.next_batch(batch_size)
    soln_pts_batched = np.zeros(input_pts.shape)
    soln_one_hot = np.zeros((batch_size, soln.shape[1], soln.shape[1]))

    #print (soln)

    for i in range(soln.shape[0]):
        soln_pts_batched[i] = input_pts[i, soln[i]]
        temp = np.zeros((soln.shape[1], soln.shape[1]))    
        temp[np.arange(soln.shape[1]), soln[i]] = 1.0
        soln_one_hot[i] = temp

    input_pts_batched = torch.transpose(torch.from_numpy(input_pts), 0, 1)    
    soln_pts_batched = torch.transpose(torch.from_numpy(soln_pts_batched), 0, 1)
    soln_one_hot_batched = torch.transpose(torch.from_numpy(soln_one_hot).float(), 0, 1)

    #print (soln_one_hot_batched)
    return input_pts_batched, soln_pts_batched, soln_one_hot_batched


In [239]:
test = DataGenerator()
batch_size=2
enc_in, dec_in, targets = test.next_batch(batch_size=batch_size, N=5)
#print (enc_in)
print (targets)

i = 0
enc_in_batch = enc_in[0]
dec_in_batch = dec_in[0]
targets_batch = targets[0]

targetss = np.zeros((batch_size, 5))

while i < batch_size-1:
    enc_in_batch = np.concatenate((enc_in_batch, enc_in[i+1]), axis=1)
    dec_in_batch = np.concatenate((dec_in_batch, dec_in[i+1]), axis=1)
    targets_batch = np.concatenate((targets_batch, targets[i+1]), axis=1)
    i+=1
    
print (targets_batch.shape)

for i in range(5):
    print (targets_batch[i])


[[[0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]]]
(5, 10)
[0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]


In [221]:
def train(enc_input, dec_input, batch_size, targets, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()                    
        
    input_length = dec_input.size()[0]
    target_length = dec_input.size()[0]

    encoder_outputs = Variable(torch.zeros(input_length, batch_size, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    loss = 0

    #Pass every token to the encoder, one at a time
    #The output is stored for when we use attention
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            enc_input[ei], encoder_hidden)
        
        #print (encoder_output.size())
        encoder_outputs[ei] = encoder_output[0]

    #decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    #decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    #The first hidden state of the decoder is the last hidden state of the encoder
    decoder_hidden = encoder_hidden

    predictions = []
    
    for di in range(target_length):
        
        if di == 0:
            decoder_input = decoder.first_step_input
        else:
            #pointers = 
            _, pointers = torch.max(decoder_attention, dim=1)
            #inputs = enc_input[pointers.data.squeeze()]
            #print (pointers.data)
            #print (enc_input)
#             print ("pointers ", pointers.data.squeeze())
#             print ("inputs", enc_input)
#             print ("enc_input T", torch.transpose(enc_input,0,1))
#             print ("print inputs pointed", torch.transpose(enc_input,0,1)[range(batch_size), pointers.data.squeeze()])
            #print (pointers.data)
            #print (enc_input.size())
            #print (pointers.size())
            #Pass the next city to the decoder as the input
            decoder_input = torch.transpose(enc_input,0,1)[range(batch_size), pointers.data.squeeze()]

        #decoder_input = dec_input[di]  # Teacher forcing
        #print (decoder_input.size())

        decoder_output, decoder_hidden, decoder_attention = \
            decoder(decoder_input, decoder_hidden, encoder_outputs, di)
    
        predictions.append(decoder_attention)
    
    #Compare the one-hot vectors with predictions
    predictions = torch.stack(predictions).squeeze()
    
#     print (predictions)
#     print (targets)
#     print (torch.transpose(targets, 0, 1))
    
    #_, index_targets = torch.max(targets, dim=1)
    #_, index_predictions = torch.max(predictions, dim=1)
    
    loss += criterion(predictions, targets)
    #loss += criterion(predictions, index_targets)

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

In [125]:
def trainIters(encoder, decoder, batch_size, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()#nn.NLLLoss()#          

    for iter in range(1, n_iters + 1):                
        enc_input, dec_input, targets = generate_tsp_batch(batch_size)
        
        targets = Variable(targets, volatile=True)
        
        loss = train(enc_input, dec_input, batch_size, targets, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

In [128]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.input_size = input_size

        #plays role of embedding of D=hidden_size
        self.embedding = nn.Linear(input_size, hidden_size)
        self.embedding.weight = torch.nn.init.xavier_uniform(self.embedding.weight)
        
        self.rnn = nn.LSTM(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)

        
    #Input contains the sequence at time t for the mini-batch
    def forward(self, input, hidden):
        
        embedded = self.embedding(Variable(input.float())).view(1, self.batch_size, self.hidden_size)
        embedded = self.dropout(embedded)  
        
        #embedded = Variable(input.float()).view(1, self.batch_size, self.input_size)       
        output, hidden = self.rnn(embedded, hidden)  
        
        return output, hidden

    def initHidden(self):
        #result = Variable(torch.zeros(1, 1, self.hidden_size))
        result = (Variable(torch.zeros(1, self.batch_size, self.hidden_size)),
          Variable(torch.zeros((1, self.batch_size, self.hidden_size))))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [222]:
class PtrDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, batch_size, sequence_length=10, dropout_p=0.1):
        super(PtrDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size        
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        
        self.attn2 = nn.Linear(self.hidden_size * 2, self.hidden_size) #as in the paper
        self.attn2.weight = torch.nn.init.xavier_uniform(self.attn2.weight)
                
        self.rnn = nn.LSTM(self.hidden_size, self.hidden_size)       
        self.V = nn.Parameter(torch.rand(self.hidden_size, 1))
        self.dropout = nn.Dropout(dropout_p)
        
        #plays role of embedding of D=hidden_size
        self.embedding = nn.Linear(output_size, hidden_size)
        self.embedding.weight = torch.nn.init.xavier_uniform(self.embedding.weight)
        
        self.first_step_input = nn.Parameter(torch.rand(self.hidden_size))

    def forward(self, input, hidden, encoder_outputs, di):                
                
        #embedded = Variable(input.float()).view(1, self.batch_size, self.output_size)    
        #Input to the first decoder step is learned
        if di == 0:
            embedded = self.first_step_input.repeat(self.batch_size).view(1, self.batch_size, self.hidden_size)
        else:
            embedded = self.embedding(Variable(input.float())).view(1, self.batch_size, self.hidden_size)
        
        embedded = self.dropout(embedded)  
        
        repeated_hidden = hidden[0].repeat(self.sequence_length, 1, 1)

        input_alignment = torch.cat((repeated_hidden, encoder_outputs), 2)                
        
        non_linearity = F.tanh(self.attn2(input_alignment)).squeeze()                           
                
        to_softmax = torch.matmul(non_linearity, self.V)
        
        #Compute the attention weights as in Dima's paper
        es = F.softmax(to_softmax, dim = 0)
        attn_weights = torch.transpose(es, 0, 1)                      
        
        output, hidden = self.rnn(embedded, hidden)            
                
        return output, hidden, attn_weights

    def initHidden(self):
        #1 because we are doing one line of the sequence at a time
        result = (Variable(torch.zeros(1, self.batch_size, self.hidden_size)),
            Variable(torch.zeros((1, self.batch_size, self.hidden_size))))
        
        if use_cuda:
            return result.cuda()
        else:
            return result

In [ ]:
hidden_size = 128
batch_size = 64 #8 works well
encoder3 = EncoderRNN(input_size=2, hidden_size=hidden_size, batch_size=batch_size)
attn_decoder3 = PtrDecoderRNN(hidden_size=hidden_size, output_size=2, batch_size=batch_size)

# if use_cuda:
#     encoder2 = encoder1.cuda()
#     attn_decoder2 = attn_decoder1.cuda()

#trainIters(encoder, attn_decoder, 50000, print_every=1000, learning_rate=0.0001) #For NLLoss
#For MSELoss:0.001, 0.0001, 0.00001 after 50,000 updates
trainIters(encoder3, attn_decoder3, batch_size, 10000, print_every=100, learning_rate=0.001) 
#p, t = evaluate(encoder, attn_decoder, 1)
#10,000 updates -> 0.007 from 0.0084

#print (np.round(p.data.numpy(),1), t)

# _, index_target = torch.max(t, dim=1)
# _, index_prediction = torch.max(p, dim=1)
# print (index_target)
# print (index_prediction)

1m 35s (- 158m 16s) (100 1%) 0.0081
3m 3s (- 149m 27s) (200 2%) 0.0078
4m 31s (- 146m 3s) (300 3%) 0.0074
5m 57s (- 143m 7s) (400 4%) 0.0072
7m 23s (- 140m 30s) (500 5%) 0.0071
9m 0s (- 141m 8s) (600 6%) 0.0070
10m 28s (- 139m 13s) (700 7%) 0.0070
11m 54s (- 136m 59s) (800 8%) 0.0069
13m 21s (- 135m 7s) (900 9%) 0.0069
14m 57s (- 134m 35s) (1000 10%) 0.0068
16m 27s (- 133m 11s) (1100 11%) 0.0068
18m 6s (- 132m 50s) (1200 12%) 0.0068
19m 45s (- 132m 15s) (1300 13%) 0.0067
21m 21s (- 131m 9s) (1400 14%) 0.0067
22m 48s (- 129m 14s) (1500 15%) 0.0067
24m 14s (- 127m 15s) (1600 16%) 0.0067
25m 40s (- 125m 21s) (1700 17%) 0.0066
27m 11s (- 123m 52s) (1800 18%) 0.0066
28m 52s (- 123m 4s) (1900 19%) 0.0066
30m 32s (- 122m 9s) (2000 20%) 0.0066
32m 15s (- 121m 19s) (2100 21%) 0.0065
33m 48s (- 119m 50s) (2200 22%) 0.0065
35m 24s (- 118m 31s) (2300 23%) 0.0065
36m 52s (- 116m 45s) (2400 24%) 0.0065
38m 23s (- 115m 9s) (2500 25%) 0.0065
39m 49s (- 113m 22s) (2600 26%) 0.0064
41m 20s (- 111m 45s) 

In [138]:
p, t = evaluate(encoder, attn_decoder, batch_size)

In [154]:
num_batches = 1
num_errors = 0.0

for j in range(num_batches):

    p, t = evaluate(encoder, attn_decoder, batch_size)
    
    p = torch.transpose(p, 0, 1)
    t = torch.transpose(t, 0, 1)
    
    for i in range(batch_size):
        #prediction = torch.transpose(p.data, 0, 1)[i].numpy().argmax(axis=1)
        #target = t.data[i].numpy().argmax(axis=1)                
        
        
        
        if i == 0:
            print (p[i].data.numpy().argmax(axis=1))
            print (t[i].numpy().argmax(axis=1))
#         if (prediction == target).all() == False:
#             print (prediction)
#             print (target)
#             print ("--")
#             num_errors +=1
        
print ("Error rate: ", num_errors/(num_batches*batch_size)*100)

[0 6 2 1 4 7 7 9 8 9]
[0 9 8 7 3 4 5 1 2 6]
Error rate:  0.0


In [135]:
def evaluate(encoder, decoder, batch_size):
    enc_input, dec_input, targets = generate_tsp_batch(batch_size)      
    
    dec_input = enc_input # we don't want to feed the answer
        
    input_length = dec_input.size()[0]
    target_length = dec_input.size()[0]

    encoder_outputs = Variable(torch.zeros(input_length, batch_size, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    encoder_hidden = encoder.initHidden()
    
    #Pass every token to the encoder, one at a time
    #The output is stored for when we use attention
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            enc_input[ei], encoder_hidden)
        
        #print (encoder_output.size())
        encoder_outputs[ei] = encoder_output[0]

    #The first hidden state of the decoder is the last hidden state of the encoder
    decoder_hidden = encoder_hidden

    predictions = []
    
    for di in range(target_length):

        decoder_input = dec_input[di]

        decoder_output, decoder_hidden, decoder_attention = \
            decoder(decoder_input, decoder_hidden, encoder_outputs)
    
        predictions.append(decoder_attention)
    
    #Compare the one-hot vectors with predictions
    predictions = torch.stack(predictions).squeeze()
    
    return predictions, targets

In [ ]:
seq_len = 3
batch_size = 1
dataset = DataGenerator()
enc_input, dec_input, targets = dataset.next_batch(batch_size, seq_len)
print("batch_size", batch_size, "seq_len", seq_len)
print("-------------encoder input-------------")
print(enc_input.shape)
print(enc_input)
print("-------------decoder input-------------")
print(dec_input.shape)
print(dec_input)
print("-------------   targets   -------------")
print(targets.shape)
print(targets)

In [138]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [Variable(torch.randn((2, 3)))
          for _ in range(5)]  # my sequence has five elements of 3 dimensions

# initialize the hidden state.
hidden = (Variable(torch.randn(1, 2, 3)),
          Variable(torch.randn((1, 2, 3))))
for i in inputs:
    
    print (i)
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 2, -1), hidden)
    
    print ("out", out)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
# inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# hidden = (Variable(torch.randn(1, 1, 3)), Variable(
#     torch.randn((1, 1, 3))))  # clean out hidden state
# out, hidden = lstm(inputs, hidden)

# print ("---")
# print(out)
# print(hidden)

Variable containing:
-0.3759  0.1582  0.8439
-1.0646 -0.2280 -0.2801
[torch.FloatTensor of size 2x3]

out Variable containing:
(0 ,.,.) = 
  0.3477  0.1061  0.0446
  0.1848 -0.0446 -0.2955
[torch.FloatTensor of size 1x2x3]

Variable containing:
 1.0165 -0.0799  0.1245
-1.4291  0.3612  1.2956
[torch.FloatTensor of size 2x3]

out Variable containing:
(0 ,.,.) = 
  0.2771  0.0416 -0.1618
  0.2204  0.2445 -0.4466
[torch.FloatTensor of size 1x2x3]

Variable containing:
 0.4057 -1.3889 -0.0718
 0.3829 -0.1459  0.7754
[torch.FloatTensor of size 2x3]

out Variable containing:
(0 ,.,.) = 
  0.2205  0.0333 -0.2364
  0.2669  0.1927 -0.3731
[torch.FloatTensor of size 1x2x3]

Variable containing:
 0.1505  0.8445  0.3930
 0.9456 -0.5531  1.7102
[torch.FloatTensor of size 2x3]

out Variable containing:
(0 ,.,.) = 
  0.3307  0.1239 -0.2493
  0.4191  0.1463 -0.4073
[torch.FloatTensor of size 1x2x3]

Variable containing:
 0.5124 -1.8215 -0.5779
-0.1179  0.4724  2.4272
[torch.FloatTensor of size 2x3]

ou